In [1]:
import re
import cv2
import os
from PIL import Image
from scipy import misc
import tensorflow as tf
import tensorflow.keras as keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import gzip
%matplotlib inline
from tensorflow.keras.layers import Input,Conv2D,MaxPool2D,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.utils import np_utils


# DATA Preprocessing

In [2]:
#Data Preprocessing

def create_dataset_train(img_folder):
   
    img_data_array=[]
    class_name=[]
    
    for file in os.listdir(img_folder):
        
        image_path = os.path.join(img_folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        
        if image.shape[2] == 3:
            image = image.mean(2)
        img_data_array.append(image)
        name_index = re.split("L|R", file)
        name_index = int(name_index[0])
        class_name.append(name_index)
        
    return np.array(img_data_array), np.array(class_name)

def create_dataset_test(img_folder):
   
    img_data_array=[]
    class_name=[]
    
    for file in os.listdir(img_folder):
        
        image_path = os.path.join(img_folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        if image.shape[2] == 3:
            image = image.mean(2)
        img_data_array.append(image)
        name_index = file.split(".")
        name_index = int(name_index[0])
        class_name.append(name_index)
        
    return np.array(img_data_array), np.array(class_name)

def normalization(image):
    image = image / image.max()
    return image


training_path = "./03_iris_training"
test_path = './03_iris_test'

train_data, train_label = create_dataset_train(training_path)
test_data, _ = create_dataset_test(test_path)

train_data = normalization(train_data)
test_data = normalization(test_data)

In [3]:
#preparing test_result_DataFrame

test_index = os.listdir(test_path)
temp = []

for index in test_index:
    index = index.split(".")
    temp.append(int(index[0]))
    
test_index = temp

In [4]:
print('Shape of Train images :',train_data.shape)
print('Shape of Train labels : ', train_label.shape)
print('Shape of Test images : ', test_data.shape)

Shape of Train images : (128, 576, 768)
Shape of Train labels :  (128,)
Shape of Test images :  (256, 576, 768)


# Model

In [5]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(576, 768)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(512, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 442368)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               113246464 
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
Total params: 113,378,048
Trainable params: 113,378,048
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(train_data, train_label, epochs=20)

Epoch 1/20
4/4 [==============================] - 4s 763ms/step - loss: 144.8325 - accuracy: 0.0078
Epoch 2/20
4/4 [==============================] - 3s 737ms/step - loss: 137.0007 - accuracy: 0.0156
Epoch 3/20
4/4 [==============================] - 3s 734ms/step - loss: 86.0499 - accuracy: 0.0078
Epoch 4/20
4/4 [==============================] - 3s 687ms/step - loss: 49.0910 - accuracy: 0.0156
Epoch 5/20
4/4 [==============================] - 3s 697ms/step - loss: 19.3501 - accuracy: 0.0000e+00
Epoch 6/20
4/4 [==============================] - 3s 679ms/step - loss: 6.2509 - accuracy: 0.0000e+00
Epoch 7/20
4/4 [==============================] - 3s 660ms/step - loss: 6.2071 - accuracy: 0.0156
Epoch 8/20
4/4 [==============================] - 3s 681ms/step - loss: 6.1525 - accuracy: 0.0156
Epoch 9/20
4/4 [==============================] - 3s 725ms/step - loss: 6.0836 - accuracy: 0.0156
Epoch 10/20
4/4 [==============================] - 3s 662ms/step - loss: 6.1169 - accuracy: 0.0078
Epoc

# Training Validation

In [8]:
predictions = model.predict(train_data)

In [9]:
print(predictions.shape)

(128, 512)


In [10]:
predict_label = []
for image in predictions:
    predict_label.append(np.argmax(image))

In [11]:
predict_label = np.array(predict_label)

In [12]:
def accuracy(original, prediction):
    accuracy = original == prediction
    accuracy = np.count_nonzero(accuracy)
    
    return accuracy / original.shape[0]
    
accuracy(train_label, predict_label)

0.015625

# Test

In [13]:
predictions = model.predict(test_data)

In [14]:
print(predictions.shape)

(256, 512)


In [15]:
test_label = []
for image in predictions:
    test_label.append(np.argmax(image))

In [16]:
test_label = np.array(test_label)

In [17]:
#결과
print(test_label)

[25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25
 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25
 25 25 25 25 25 25  3 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25
 25 25 25 25 25 25 25 25 25  9 25 25 25 25 25 25 25 25 25 25 25 25 25 25
 25 25 25 25 25 25 25 44 25 25 25 25 25 25 25 25 14 25 44 25 25  9 25 25
 25 25 25 25 25 25 25 25 25 25 25 25 14 25 14 25 25 25 25 25 25 25 25 25
 25  9 14 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 14 25 25 25 25
 25 25 25 14 25 25 25 25 25 25 25  9 25 25 25 25  9 25 25 25 25 25 25 25
 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 44 25 14
 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25  9 25 25 25 25 25 14 25 25
 25 25 25 25 25  9 25 25 25 25 25 25 25 25 25 25]


# Extract Pandas DataFrame

In [18]:
test_result_df = pd.DataFrame([x for x in zip(test_index, test_label)], columns=['Image', 'Answer'])

In [19]:
test_result_df.head()

,Image,Answer
0,1,25
1,10,25
2,100,25
3,101,25
4,102,25


In [20]:
test_result_df = test_result_df.sort_values(by="Image")
test_result_df.head()

,Image,Answer
0,1,25
111,2,25
179,3,9
190,4,25
201,5,25


In [21]:
test_result_df.to_csv("./result.csv", index=False)